<a href="https://colab.research.google.com/github/vishal-narwal/To-Do-List/blob/main/iron%20condor%20order%20execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Install or upgrade required packages
!pip install --upgrade alpaca-py python-dotenv

# ✅ Import libraries and set credentials directly (paper trading)
from alpaca.trading.client import TradingClient
from alpaca.trading.enums import OrderSide, OrderClass, TimeInForce
from alpaca.trading.requests import LimitOrderRequest, OptionLegRequest
from alpaca.data.historical import OptionHistoricalDataClient
from alpaca.data.requests import OptionLatestQuoteRequest
from alpaca.common.exceptions import APIError
import pandas as pd
import os
import time
import re

# ✅ API Keys (Paper Trading)
API_KEY = "PK98WO4ZTD4QH5GOUYHF"
API_SECRET = "O20FKkTwoE064rqfpdJ4bjgtdYQMM65Yc0vM23bW"

# Initialize trading and data clients
client = TradingClient(API_KEY, API_SECRET, paper=True)
data_client = OptionHistoricalDataClient(API_KEY, API_SECRET)


In [ ]:
# ✅ Read your formatted file (already uploaded in Colab)
df = pd.read_csv("/content/new1_formatted_like_ic3.txt", sep=r'\t+', engine='python')

# ✅ Build OCC-compliant symbol
def build_occ_symbol(symbol, expiration, strike, option_type):
    exp = pd.to_datetime(expiration).strftime('%y%m%d')
    opt_type = 'P' if option_type.lower() == 'put' else 'C'
    strike_code = f"{int(float(strike) * 1000):08d}"
    return f"{symbol.upper()}{exp}{opt_type}{strike_code}"

# ✅ Prepare request for quotes
quote_legs = []
for _, row in df.iterrows():
    for leg, otype in [('LP', 'put'), ('SP', 'put'), ('SC', 'call'), ('LC', 'call')]:
        quote_legs.append({
            'Symbol': row['Symbol'],
            'Expiration': row['Expiration'],
            'Leg': leg,
            'Strike': row[leg],
            'Type': otype,
            'OCC': build_occ_symbol(row['Symbol'], row['Expiration'], row[leg], otype)
        })

quote_df = pd.DataFrame(quote_legs)


In [ ]:
# ✅ Fetch quotes from Alpaca
all_quotes = {}
batch_size = 50
for i in range(0, len(quote_df), batch_size):
    symbols = quote_df['OCC'].iloc[i:i+batch_size].tolist()
    request = OptionLatestQuoteRequest(symbol_or_symbols=symbols)
    quotes = data_client.get_option_latest_quote(request)
    all_quotes.update(quotes)

# ✅ Attach bid, ask, mid
quote_df['Bid'] = quote_df['OCC'].map(lambda s: all_quotes.get(s).bid_price if all_quotes.get(s) else None)
quote_df['Ask'] = quote_df['OCC'].map(lambda s: all_quotes.get(s).ask_price if all_quotes.get(s) else None)
quote_df['Mid'] = quote_df.apply(lambda r: round((r['Bid'] + r['Ask']) / 2, 3) if r['Bid'] and r['Ask'] else None, axis=1)

# ✅ Compute & export Iron Condor results
output_lines = []
for symbol in df['Symbol'].unique():
    legs = quote_df[quote_df['Symbol'] == symbol].set_index('Leg')
    if {'LP', 'SP', 'SC', 'LC'}.issubset(legs.index):
        try:
            output_lines.append("\n--- Iron Condor Bid / Ask / Mid Prices ---")
            for leg in ['LP', 'SP', 'SC', 'LC']:
                line = f"{legs.loc[leg, 'OCC']} ({leg}) → Bid: {legs.loc[leg, 'Bid']}, Mid: {legs.loc[leg, 'Mid']}, Ask: {legs.loc[leg, 'Ask']}"
                output_lines.append(line)

            lp_mid, lc_mid = legs.loc['LP', 'Mid'], legs.loc['LC', 'Mid']
            sp_mid, sc_mid = legs.loc['SP', 'Mid'], legs.loc['SC', 'Mid']
            lp_bid, lc_bid = legs.loc['LP', 'Bid'], legs.loc['LC', 'Bid']
            sp_bid, sc_bid = legs.loc['SP', 'Bid'], legs.loc['SC', 'Bid']
            lp_ask, lc_ask = legs.loc['LP', 'Ask'], legs.loc['LC', 'Ask']
            sp_ask, sc_ask = legs.loc['SP', 'Ask'], legs.loc['SC', 'Ask']

            combined_bid = round(sc_bid + sp_bid - lp_ask - lc_ask, 2)
            combined_ask = round(sc_ask + sp_ask - lp_bid - lc_bid, 2)
            combined_mid = round((sp_mid + sc_mid) - (lp_mid + lc_mid), 2)

            output_lines.append(f"\nSymbol | Expiration | CombinedBid | CombinedMid | CombinedAsk")
            output_lines.append(f"{symbol:<7} {legs.loc['LP', 'Expiration']}     {combined_bid:<12} {combined_mid:<12} {combined_ask}")
        except:
            continue

# ✅ Save to text file
with open("/content/iron_condor_results.txt", "w") as f:
    for line in output_lines:
        f.write(line + "\n")


In [ ]:
# ✅ Parse result file
def parse_condor_file(filepath):
    with open(filepath, 'r') as f:
        blocks = f.read().split('--- Iron Condor Bid / Ask / Mid Prices ---')[1:]
    condors = []
    for block in blocks:
        lines = block.strip().splitlines()
        legs = {}
        for i in range(4):
            m = re.match(r"(\w+)\s+\((\w+)\).*Bid: ([\d.]+), Mid: ([\d.]+), Ask: ([\d.]+)", lines[i])
            if m:
                sym, leg, bid, mid, ask = m.groups()
                legs[leg] = {
                    'symbol': sym,
                    'bid': float(bid),
                    'mid': float(mid),
                    'ask': float(ask),
                    'side': 'sell' if leg in ['SP', 'SC'] else 'buy'
                }
        summary = lines[-1]
        s = re.match(r"(\w+)\s+([\d/]+)\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)", summary)
        if s:
            condors.append({
                'symbol': s[1],
                'expiration': s[2],
                'combined_bid': float(s[3]),
                'combined_mid': float(s[4]),
                'combined_ask': float(s[5]),
                'legs': legs
            })
    return condors

# ✅ Submit multi-leg order
def place_mleg_order(legs, limit_price, action='sell', qty=1): # Added qty parameter with default value 1
    order_legs = []
    for leg_code in ['LP', 'SP', 'SC', 'LC']:
        leg = legs[leg_code]
        side = OrderSide.BUY if (action == 'buy' and leg['side'] == 'buy') or (action == 'sell' and leg['side'] == 'buy') else OrderSide.SELL
        order_legs.append(OptionLegRequest(symbol=leg['symbol'], side=side, ratio_qty=1))
    try:
        return client.submit_order(LimitOrderRequest(
            order_class=OrderClass.MLEG,
            time_in_force=TimeInForce.DAY,
            limit_price=round(limit_price, 2),
            legs=order_legs,
            qty=qty # Added qty to LimitOrderRequest
        ))
    except APIError as e:
        print(f"❌ Error: {e}")
        return None

# ✅ Wait for fill
def wait_for_order_fill(order_id, wait_seconds=1):
    for _ in range(wait_seconds):
        time.sleep(1)
        order = client.get_order_by_id(order_id)
        if order.status == "filled":
            return True
    return False

# ✅ Execute each iron condor
condors = parse_condor_file("/content/iron_condor_results.txt")
for i, item in enumerate(condors):
    print(f"\n🔁 [{i+1}/{len(condors)}] {item['symbol']}")

    price = item['combined_ask']
    min_price = round(item['combined_mid'] * 1.05, 2)
    filled = False

    # Submit SELL MLEG
    for attempt in range(6):
        order = place_mleg_order(item['legs'], price, 'sell')
        if order:
            print(f"🟡 SELL submitted @ {price}")
            if wait_for_order_fill(order.id, 5):
                print(f"✅ SELL filled @ {price}")
                filled = True
                break
        price = max(min_price, round(price - 0.01, 2))
        time.sleep(1)

    if not filled:
        print(f"⏸️ SELL not filled. Leaving open at {price}")
        continue

    # Submit BUY MLEG
    price = item['combined_bid']
    while price <= item['combined_mid']:
        order = place_mleg_order(item['legs'], price, 'buy')
        if order:
            print(f"🟢 BUY submitted @ {price}")
            if wait_for_order_fill(order.id, 1):
                print(f"✅ BUY filled @ {price}")
                break
        price = round(price + 0.01, 2)
        time.sleep(1)



🔁 [1/40] LYFT
🟡 SELL submitted @ 0.32
🟡 SELL submitted @ 0.31
🟡 SELL submitted @ 0.3
🟡 SELL submitted @ 0.29
🟡 SELL submitted @ 0.28
🟡 SELL submitted @ 0.27
⏸️ SELL not filled. Leaving open at 0.26

🔁 [2/40] PINS
🟡 SELL submitted @ 0.56
🟡 SELL submitted @ 0.55
🟡 SELL submitted @ 0.54
🟡 SELL submitted @ 0.53
🟡 SELL submitted @ 0.52
🟡 SELL submitted @ 0.51
⏸️ SELL not filled. Leaving open at 0.5

🔁 [3/40] GRPN
🟡 SELL submitted @ 1.19
🟡 SELL submitted @ 1.18
🟡 SELL submitted @ 1.17
🟡 SELL submitted @ 1.16
🟡 SELL submitted @ 1.15
🟡 SELL submitted @ 1.14
⏸️ SELL not filled. Leaving open at 1.13

🔁 [4/40] DKNG
🟡 SELL submitted @ 0.68
🟡 SELL submitted @ 0.67
🟡 SELL submitted @ 0.66
🟡 SELL submitted @ 0.65
🟡 SELL submitted @ 0.64
🟡 SELL submitted @ 0.63
⏸️ SELL not filled. Leaving open at 0.62

🔁 [5/40] OSCR
🟡 SELL submitted @ 0.56
🟡 SELL submitted @ 0.55
🟡 SELL submitted @ 0.54
🟡 SELL submitted @ 0.53
🟡 SELL submitted @ 0.52
🟡 SELL submitted @ 0.51
⏸️ SELL not filled. Leaving open at 0.5

🔁 